https://hub.docker.com/r/sath89/oracle-12c/

https://hub.docker.com/_/mysql/

https://stackoverflow.com/questions/8858008/how-to-move-a-file-in-python

In [1]:
# encoding=utf8
# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

import pandas as pd
import os
from time import time, ctime

import concurrent.futures

from lxml import etree
import json
import re


In [2]:
!ls

add_bidder	    boss_urls.csv		 pic_down_func-add2.ipynb
addbidderPics	    data			 pic_down_func_bidder.ipynb
addbidder_urls.csv  emp_pages4			 pic_down_func.ipynb
addbossPics	    freelancer.py		 picDown.py
addboss_urls.csv    headPics			 picDown.pyc
add_owner	    old				 pic_down_report.ipynb
addtional	    Parse_html_Stat-Copy1.ipynb  pic_report.ipynb
bidderPics	    Parse_html_Stat.ipynb	 PicsWithFace.zip
bidder_urls.csv     Parse_html_V5.ipynb		 recIpython
bid_pages2	    parse.ipynb			 Tensorflow
bossPics	    pic_down_func-add1.ipynb


In [3]:
!pwd

/home/soyoung/face/Freelancer


In [4]:
# path='technosystem.html'

folder='bid_pages2/'
cont = os.listdir(folder)
len(cont)

68227

In [5]:
def flatten_dict(data, layers=1, drop_deeper=False):

        for _ in range(layers):
            data = [(k, v) if not isinstance(v, dict) else [(k + '.' + k2, v2) for k2, v2 in v.items()] for k, v in
                    data.items()]
            data = [item for sublist in data for item in sublist if isinstance(sublist, list)] + [y for y in data if
                                                                                                  not isinstance(y,
                                                                                                                 list)]
            data = dict(data)

        if drop_deeper:
            data = {k: v for k, v in data.items() if not isinstance(v, dict) or isinstance(v, list)}

        return data

In [6]:
failed_names=[]

def get_columns(path):
    global failed_names
    get=[]
    
    with open(folder+path,'r') as fl:
        text=fl.read()
        tree=etree.HTML(text)
    try:
        script = tree.xpath('//script')[-1]
        data = re.search(r"__export\('profileServerData',(.*?)\n", script.text)        
        jsondata=data.group(1)[:-2]
        
        json_dict = json.loads(jsondata)
        flat = flatten_dict(json_dict, layers=1)
        names = flat.keys()
        return names
    except Exception as e:
        failed_names.append(path)
        return []

    

In [7]:
from collections import defaultdict
stat = defaultdict(int)

In [8]:
%%time
for i in cont:
    for k in get_columns(i):
        stat[k]+=1
len(stat),len(failed_names)

CPU times: user 5min 3s, sys: 4.29 s, total: 5min 7s
Wall time: 5min 48s


In [9]:
len(stat),len(failed_names)

(73, 13077)

In [10]:
col = pd.Series(stat.values(), index=stat.keys())
col.shape

(73,)

In [11]:
col.value_counts()

55150    72
42643     1
dtype: int64

In [13]:
col.sort_values(ascending=True).head()

hourlyRate               42643
about.support_status     55150
about.mobile_tracking    55150
about.timezone           55150
about.id                 55150
dtype: int64

In [14]:
print failed_names[:3]
len(failed_names)

['np101.html', 'Waqas0001.html', 'acmaceda.html']


13077

In [15]:
try:
    os.mkdir(folder+'NoJson')
except:
    print 'exists'

In [16]:
!ls {folder}'NoJson'|wc

      0       0       0


In [17]:
import shutil

In [18]:
%%time
# os.rename("path/to/current/file.foo", "path/to/new/desination/for/file.foo")
for i in failed_names:
    try:
        shutil.move(folder+i, folder+'NoJson/')
    except:
        pass


CPU times: user 140 ms, sys: 184 ms, total: 324 ms
Wall time: 599 ms


In [19]:
!ls {folder}'NoJson'|wc

  13077   13077  204811


In [20]:
!ls {folder}|wc

  55151   55151  857809


In [21]:
col.to_csv('bidder1.csv', header=['count'], index_label='name')

In [23]:
df1 = pd.read_csv('bidder1.csv').set_index('name')
df2 = pd.read_csv('boss1.csv').set_index('name')

In [26]:
df1.shape, df2.shape

((73, 1), (73, 1))

In [25]:
df1+df2

,count
name,
abTests,66530
about.account_balances,66529
about.address,66529
about.avatar,66529
about.avatar_cdn,66529
about.avatar_large,66529
about.avatar_large_cdn,66529
about.badges,66529
about.chosen_role,66529


In [68]:
# group=[]
# def batch(batchfiles):
#     global group
#     group=[]
#     start=time()
#     with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
#         executor.map(get_single, batchfiles)
#     past='{:.2f}'.format(time()-start)
#     now=ctime()[4:]

#     print 'profiles ',len(batchfiles),'  used ',past,'s', now
#     return group


In [74]:
len(batch(cont[:2]))

profiles  2   used  0.02 s Sep 28 01:07:10 2017


2

In [84]:
len(batch(cont[:10])[6])

'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
profiles  10   used  0.06 s Sep 28 01:08:03 2017


168